# Introducción

Megaline busca modernizar su oferta migrando a clientes de planes heredados a los nuevos planes Smart y Ultra. Con datos de suscriptores que ya hicieron el cambio, el objetivo es desarrollar un modelo de clasificación que recomiende el plan más adecuado para cada cliente, alcanzando al menos un 75% de exactitud.

## Abre y examina el archivo de datos. Dirección al archivo:/datasets/users_behavior.csv

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor




In [13]:
df = pd.read_csv('users_behavior.csv')

print(df.shape)
print(df.head(10))

(3214, 5)
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0


## Segmentación de los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [14]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

df_train, df_valid = train_test_split(df, test_size=0.25, random_state=54321) # segmenta el 25% de los datos para hacer el conjunto de validación
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

test_df = pd.read_csv('users_behavior.csv')
test_features = test_df.drop(['is_ultra'], axis=1)
test_target = test_df['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)
(2410,)
(804, 4)
(804,)


## Investigación de la calidad de diferentes modelos cambiando los hiperparámetros. Descripción breve de los hallazgos del estudio

### Random Forest (comprobando la calidad del modelo utilizando el conjunto de prueba)

- Este código recorre diferentes valores del parámetro n_estimators (número de árboles) del RandomForestClassifier, entrenando un modelo para cada valor de 1 a 20 y evaluándolo en el conjunto de validación. Se guarda el número de árboles que genera la mayor exactitud (accuracy) en el conjunto de validación.

- El resultado indica que, al evaluar diferentes modelos RandomForest con entre 1 y 19 árboles, el modelo que alcanzó la mayor exactitud en el conjunto de validación utiliza 16 árboles (n_estimators = 16) y logra una exactitud de aproximadamente 80.22%. Esto significa que este modelo clasifica correctamente alrededor del 80% de las instancias en el conjunto de validación.

- En resumen, el RandomForest con 16 estimadores es el que ofrece el mejor rendimiento en validación en este experimento, y su exactitud del 80.22% es una medida de cuán efectivo es el modelo para predecir correctamente la variable objetivo en datos nuevos.

In [15]:
best_score = 0
best_est = 0
for est in range(1, 21): 
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    model.fit(features_train, target_train) 
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score 
        best_est = est 

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 16): 0.8022388059701493


### Regresión logística

- Precisión en entrenamiento (71.8%): El modelo clasifica correctamente aproximadamente el 72% de los casos en el conjunto de entrenamiento.

- Precisión en validación (69.4%): En datos nuevos (validación), el modelo logra una precisión del 69.4%, lo que sugiere que su capacidad de generalización es razonable, ya que la diferencia con el entrenamiento es de solo unos 2 puntos porcentuales.

- La diferencia pequeña entre el entrenamiento y la validación sugiere que no hay un sobreajuste muy pronunciado.

- Una precisión cercana al 70% indica que el modelo clasifica correctamente aproximadamente 7 de cada 10 instancias, lo cual es aceptable, pero puede haber margen de mejora.

In [16]:
model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7178423236514523
Accuracy del modelo de regresión logística en el conjunto de validación: 0.6940298507462687


### Arboles de Desición

- Con max_depth = 1, el árbol obtiene un 75.8% en entrenamiento, y a medida que se incrementa la profundidad hasta 10, la exactitud de entrenamiento sube a 87.1%. Esto indica que modelos más complejos se ajustan mejor a los datos de entrenamiento.

- La exactitud en el conjunto de prueba empieza en 73.6% para max_depth = 1 y aumenta gradualmente hasta alcanzar alrededor de 79.5% con max_depth = 8. Sin embargo, a partir de ese punto (max_depth = 9 y 10), la exactitud de prueba se mantiene casi igual o incluso disminuye levemente (79.2% y 79.1%, respectivamente).

- En conclusión aunque aumentar la profundidad mejora el ajuste sobre los datos de entrenamiento, la mejora en el conjunto de prueba se estabiliza alrededor de max_depth = 8, lo que sugiere que ese valor podría ser óptimo. Más allá de esa profundidad, el incremento en la complejidad no se traduce en un mejor desempeño en datos no vistos, e incluso puede perjudicarlo por el sobreajuste.

In [17]:
features_train, test_features, target_train, test_target = train_test_split(features, target, test_size=0.25, random_state=54321)
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    model.fit(features_train, target_train)
    train_predictions = model.predict(features_train)
    test_predictions = model.predict(test_features)
    train_accuracy = accuracy_score(target_train, train_predictions)
    test_accuracy = accuracy_score(test_target, test_predictions)
    

    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", train_accuracy)
    print("Conjunto de prueba:", test_accuracy)
    print()

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.7580912863070539
Conjunto de prueba: 0.736318407960199

Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.7871369294605809
Conjunto de prueba: 0.7699004975124378

Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.8024896265560166
Conjunto de prueba: 0.7810945273631841

Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.8116182572614108
Conjunto de prueba: 0.7810945273631841

Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.816597510373444
Conjunto de prueba: 0.7810945273631841

Exactitud de max_depth igual a 6
Conjunto de entrenamiento: 0.8244813278008298
Conjunto de prueba: 0.7835820895522388

Exactitud de max_depth igual a 7
Conjunto de entrenamiento: 0.8369294605809129
Conjunto de prueba: 0.7873134328358209

Exactitud de max_depth igual a 8
Conjunto de entrenamiento: 0.8526970954356846
Conjunto de prueba: 0.7947761194029851

Exactitud de max_depth igual a 9
Conjunto de entre

En los siguientes codigos realizaremos una búsqueda de hiperparámetro para seleccionar el mejor modelo de regresión basado en un árbol de decisión y un random forest, variando el parámetro max_depth (profundidad máxima) entre 1 y 10. La idea es entrenar varios modelos con distintas profundidades y evaluar cuál tiene el menor error en el conjunto de validación, utilizando el error cuadrático medio (EMC).

In [18]:
best_model = None
best_result = 100000
best_depth = 0
for depth in range(1, 11): 
    model = DecisionTreeRegressor(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result = mean_squared_error(target_valid, predictions_valid)**0.5 
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

RECM del mejor modelo en el conjunto de validación (max_depth = 6): 0.3999335684302242


El resultado indica que, al ajustar el hiperparámetro de profundidad máxima, el mejor modelo en términos de RECM (Raíz del Error Cuadrático Medio) en el conjunto de validación es un árbol de decisión con una profundidad máxima de 6. Con este modelo, la RECM obtenida fue aproximadamente 0.39993, lo que significa que, en promedio, la diferencia entre los valores predichos y los valores reales en el conjunto de validación es de cerca de 0.4 unidades.

In [19]:
best_error = 100 
best_est = 0
best_depth = 0
for depth in range (1, 11):
    model = RandomForestRegressor(random_state=54321, n_estimators=est, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    error = mean_squared_error(target_valid, predictions_valid)**0.5
    if error < best_error: 
        best_error = error
        best_est = est
        best_depth = depth

print("RECM del mejor modelo en el conjunto de validación:", best_error, "n_estimators:", best_est, "best_depth:", best_depth)

RECM del mejor modelo en el conjunto de validación: 0.3838275787804249 n_estimators: 20 best_depth: 9


El resultado indica que, al ajustar un modelo RandomForestRegressor y explorar combinaciones de hiperparámetros, el mejor desempeño en el conjunto de validación se alcanzó con un modelo que utiliza:

20 estimadores (n_estimators = 20): Es decir, el modelo utiliza 20 árboles en el ensamble.
Máxima profundidad (max_depth) de 9: Cada árbol se limita a una profundidad máxima de 9 niveles.
Con esta configuración, el modelo obtuvo una RECM (raíz del error cuadrático medio) de aproximadamente 0.3838 en el conjunto de validación. Esto significa que, en promedio, las predicciones del modelo se desvían de los valores reales en alrededor de 0.38 unidades.

## CONCLUSIÓN 

- Un RECM menor indica mejores predicciones en promedio. En este caso, el Random Forest tiene un error ligeramente inferior, lo que sugiere que sus predicciones se desvían menos de los valores reales.

- En resumen, aunque la diferencia en RECM es modesta, el modelo RandomForestRegressor es superior, ya que ofrece un menor error de predicción y, en general, se comporta mejor en términos de estabilidad y capacidad de generalización.